In [1]:
pip install opencv-python

   ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
   --- ------------------------------------ 1.0/12.9 MB 4.6 MB/s eta 0:00:03
   ---------- ----------------------------- 3.4/12.9 MB 7.7 MB/s eta 0:00:02
   ----------------- ---------------------- 5.5/12.9 MB 8.4 MB/s eta 0:00:01
   --------------------- ------------------ 6.8/12.9 MB 8.4 MB/s eta 0:00:01
   ------------------------ --------------- 7.9/12.9 MB 7.5 MB/s eta 0:00:01
   ------------------------- -------------- 8.1/12.9 MB 6.2 MB/s eta 0:00:01
   --------------------------- ------------ 8.9/12.9 MB 6.0 MB/s eta 0:00:01
   ------------------------------ --------- 10.0/12.9 MB 5.7 MB/s eta 0:00:01
   ---------------------------------- ----- 11.0/12.9 MB 5.6 MB/s eta 0:00:01
   ------------------------------------ --- 11.8/12.9 MB 5.6 MB/s eta 0:00:01
   -------------------------------------- - 12.3/12.9 MB 5.3 MB/s eta 0:00:01
   ---------------------------------------  12.8/12.9 MB 4.9 MB/s eta 0:00:01
 

In [2]:
pip install mediapipe

  Using cached absl_py-2.3.1-py3-none-any.whl.metadata (3.3 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached jax-0.6.2-py3-none-any.whl.metadata (13 kB)
  Using cached opencv_contrib_python-4.12.0.88-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached protobuf-4.25.8-cp310-abi3-win_amd64.whl.metadata (541 bytes)
  Using cached sounddevice-0.5.2-py3-none-win_amd64.whl.metadata (1.6 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached pyparsing-3.2.3-py3-none-any.whl.metadata (5.0 kB)
INFO: pip is looking at multiple versions of opencv-contrib-python to determine which version is compatible with other requirements. This could take a while.
  Using cached opencv_contrib_python-4.11.0.86-cp37-abi3-win_amd64.whl.metadata (20 kB)
   ---------------------------------------- 0.0/51.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.


In [5]:
import cv2
import numpy as np
import PoseModule as pm  # Your poseDetector class

def main():
    cap = cv2.VideoCapture(0)
    detector = pm.poseDetector()

    rep_count = 0
    stage = None  # Track if user is "up" or "down" in the lunge
    feedback = "Get Ready!"

    # For smoothing angle to avoid jitter
    angle_history = []

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Can't receive frame (stream end?). Exiting ...")
            break

        frame = detector.findPose(frame, draw=True)
        lmList = detector.findPosition(frame, draw=False)

        if lmList:
            # Right leg landmarks IDs
            hip_id, knee_id, ankle_id = 24, 26, 28

            angle = detector.findAngle(frame, hip_id, knee_id, ankle_id)

            # Smooth angle over last 5 frames
            angle_history.append(angle)
            if len(angle_history) > 5:
                angle_history.pop(0)
            smooth_angle = sum(angle_history) / len(angle_history)

            # Rep counting logic
            if smooth_angle > 160:
                if stage != "up":
                    stage = "up"
                    feedback = "Stand Tall"
            elif smooth_angle < 90:
                if stage == "up":
                    stage = "down"
                    rep_count += 1
                    feedback = "Good Lunge!"
            else:
                feedback = "Keep Going"

            # Calculate depth % for progress bar (0% shallow, 100% deep)
            max_angle = 170
            min_angle = 80
            depth_percent = max(0, min(100, int((max_angle - smooth_angle) / (max_angle - min_angle) * 100)))

            # Draw progress bar
            bar_x, bar_y = 50, 100
            bar_width, bar_height = 30, 300

            filled_height = int((depth_percent / 100) * bar_height)
            # Bar background
            cv2.rectangle(frame, (bar_x, bar_y), (bar_x + bar_width, bar_y + bar_height), (200, 200, 200), 2)
            # Filled portion
            cv2.rectangle(frame, (bar_x, bar_y + bar_height - filled_height), 
                          (bar_x + bar_width, bar_y + bar_height), (0, 255, 0), -1)
            # Depth percent text
            cv2.putText(frame, f"{depth_percent}%", (bar_x - 10, bar_y + bar_height + 40),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

            # Show rep count and feedback on screen
            cv2.putText(frame, f"Lunges: {rep_count}", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 2,
                        (255, 0, 0), 3)
            cv2.putText(frame, feedback, (10, 130), cv2.FONT_HERSHEY_SIMPLEX, 1,
                        (0, 255, 0), 2)
            cv2.putText(frame, f"Knee Angle: {int(smooth_angle)}", (10, 180), cv2.FONT_HERSHEY_SIMPLEX, 1,
                        (0, 255, 255), 2)

        else:
            feedback = "Step into view"
            cv2.putText(frame, feedback, (10, 130), cv2.FONT_HERSHEY_SIMPLEX, 1,
                        (0, 0, 255), 2)

        cv2.imshow("Lunge Counter", frame)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()
